In [1]:
source = '~/Downloads/Julia/NM_LEVEL2_threshold.xlsx'
threshold = 100
cell_color = r'#009933'

In [2]:
#!{sys.executable} -m pip install numpy pandas tabulate openpyxl xlsxwriter
!python3 -m pip install numpy pandas tabulate openpyxl xlsxwriter xlrd

In [3]:
import pandas as pd
import numpy as np
from IPython.display import display
from tabulate import tabulate

In [14]:
r = pd.ExcelFile(source)

sheet_names = r.sheet_names

data = []

for animal in sheet_names:
    data.append(pd.read_excel(source, sheet_name=animal))

In [15]:
display(data[0])

,Region,Channel,ROI,Area,Mean,Min,Max
0,RIGHT CORE 1,DRD1,36,90.766,0.000,0,0
1,RIGHT CORE 1,DRD1,72,117.803,0.000,0,0
2,RIGHT CORE 1,DRD1,138,70.295,0.000,0,0
3,RIGHT CORE 1,DRD1,11,63.343,0.207,0,14
4,RIGHT CORE 1,DRD1,86,92.697,0.275,0,41
...,...,...,...,...,...,...,...
3478,LEFT SHELL 3,OXTR,98,298.948,2.584,0,1319
3479,LEFT SHELL 3,OXTR,107,188.098,9.316,0,1363
3480,LEFT SHELL 3,OXTR,113,76.861,39.387,0,1448
3481,LEFT SHELL 3,OXTR,84,151.405,8.671,0,1629


In [29]:
sorted_data = []
for sheet in data:
    sorted_data.append((sheet.sort_values(by=['Region','Channel','Max'])))

display(sorted_data)

[             Region Channel  ROI     Area    Mean  Min   Max
 2067    LEFT CORE 1    DRD1    3   69.523   0.000    0     0
 2068    LEFT CORE 1    DRD1    4  106.988   0.000    0     0
 2069    LEFT CORE 1    DRD1    8   64.116   0.000    0     0
 2070    LEFT CORE 1    DRD1    9   60.253   0.000    0     0
 2071    LEFT CORE 1    DRD1   11   98.104   0.000    0     0
 ...             ...     ...  ...      ...     ...  ...   ...
 2062  RIGHT SHELL 3    OXTR   72   64.116   6.440    0   785
 2063  RIGHT SHELL 3    OXTR  100   91.152   5.992    0   838
 2064  RIGHT SHELL 3    OXTR  118  150.633   5.287    0   924
 2065  RIGHT SHELL 3    OXTR   46  222.473   8.714    0  1122
 2066  RIGHT SHELL 3    OXTR   53  149.474  30.196    0  3264
 
 [3483 rows x 7 columns],
              Region Channel  ROI     Area     Mean Min   Max
 2209    LEFT CORE 1    DRD1    8   67.205        0   0     0
 2254    LEFT CORE 1    DRD1   53   86.131        0   0     0
 2262    LEFT CORE 1    DRD1   61   61.026

In [30]:
display(sorted_data[0])

,Region,Channel,ROI,Area,Mean,Min,Max
2067,LEFT CORE 1,DRD1,3,69.523,0.000,0,0
2068,LEFT CORE 1,DRD1,4,106.988,0.000,0,0
2069,LEFT CORE 1,DRD1,8,64.116,0.000,0,0
2070,LEFT CORE 1,DRD1,9,60.253,0.000,0,0
2071,LEFT CORE 1,DRD1,11,98.104,0.000,0,0
...,...,...,...,...,...,...,...
2062,RIGHT SHELL 3,OXTR,72,64.116,6.440,0,785
2063,RIGHT SHELL 3,OXTR,100,91.152,5.992,0,838
2064,RIGHT SHELL 3,OXTR,118,150.633,5.287,0,924
2065,RIGHT SHELL 3,OXTR,46,222.473,8.714,0,1122


In [31]:
for sheet in sorted_data:
    sheet['Max_Delta'] = sheet.groupby(['Region','Channel'])['Max'].diff()

display(sorted_data)

ValueError: 

In [34]:
display(sorted_data[0])

,Region,Channel,ROI,Area,Mean,Min,Max,Max_Delta
2067,LEFT CORE 1,DRD1,3,69.523,0.000,0,0,NaN
2068,LEFT CORE 1,DRD1,4,106.988,0.000,0,0,0.0
2069,LEFT CORE 1,DRD1,8,64.116,0.000,0,0,0.0
2070,LEFT CORE 1,DRD1,9,60.253,0.000,0,0,0.0
2071,LEFT CORE 1,DRD1,11,98.104,0.000,0,0,0.0
...,...,...,...,...,...,...,...,...
2062,RIGHT SHELL 3,OXTR,72,64.116,6.440,0,785,99.0
2063,RIGHT SHELL 3,OXTR,100,91.152,5.992,0,838,53.0
2064,RIGHT SHELL 3,OXTR,118,150.633,5.287,0,924,86.0
2065,RIGHT SHELL 3,OXTR,46,222.473,8.714,0,1122,198.0


In [ ]:
first_threshold_table = sorted_data.query('Max_Delta > 100').groupby(['Region','Channel']).first()
first_threshold_table['Max_Delta_First_Hit'] = True
display(first_threshold_table)

In [ ]:
merged_data = sorted_data.reset_index().merge(first_threshold_table, how='left')
display(merged_data)

In [ ]:
def style_when_true(series):
    match_table = [1 if x == True else 0 for x in series]
    return [f'background-color: {cell_color}' if v else '' for v in match_table]

In [ ]:
formatted_data = merged_data.style.apply(style_when_true, subset='Max_Delta_First_Hit')
display(formatted_data)

In [2]:
with pd.ExcelWriter(f"{sample}.xlsx", engine='xlsxwriter') as writer:
    first_threshold_table.to_excel(writer, sheet_name=f'{sample}-summary')
    formatted_data.to_excel(writer, sheet_name=sample, index=True)
    
    #Autofit columns
    for sheetname, df in writer .sheets.items():
        worksheet = writer.sheets[sheetname]
        worksheet.set_column(0, 10, 20)

NameError: name 'pd' is not defined